# BP Algorithm
谢昊君 1200015556 元培学院

# k层BP 算法推导
$W^{(k)}$表示k-1层到第k层的权重矩阵,$z^i=f(W^i\cdot a^{i-1}+b)$,或每次将$a$加入单位向量，使原式变为$z^i =[f(W^i\cdot a^{i-1});1]$

$$a^{(i+1)}=f(z^{(i+1)})$$

激活函数满足$f'(z)=f(z)(1-f(z))$

$h_{W,b}(x)$表示该神经网络对x特征的预测

损失函数为：
$$J(W,b; x,y) = \frac{1}{2} \left\| h_{W,b}(x) - y \right\|^2.$$
对m个输入样本而言，总损失函数为:
$$J(W,b)=\frac{1}{m}\sum_i ^m J(W,b;x^{(i)},y^{(i)})$$

对该损失函数按梯度最优化更新权重矩阵$W$，即
\begin{align}
W_{ij}^{(l)} &= W_{ij}^{(l)} - \alpha \frac{\partial}{\partial W_{ij}^{(l)}} J(W,b) \\
b_{i}^{(l)} &= b_{i}^{(l)} - \alpha \frac{\partial}{\partial b_{i}^{(l)}} J(W,b)
\end{align}

接下来求$\frac{\partial}{\partial W_{ij}^{(l)}} J(W,b) $.

首先考虑损失函数对神经元的导数,每一个神经元对loss function的影响/贡献,定义$\delta_i ^k$ 为第k层第i个神经元对loss function的贡献

于是有
$$\delta_i ^k = \frac{\partial}{\partial z_i ^k} \frac{1}{2} \left\| h_{W,b}(x) - y \right\|^2 = - (y_i - a^{(n_l)}_i) \cdot f'(z^{(n_l)}_i)
$$

继续向内层求导则有：

$$\delta^{(l)}_i = \left( \sum_{j=1}^{s_{l+1}} W^{(l)}_{ji} \delta^{(l+1)}_j \right) f'(z^{(l)}_i)$$

于是根据链式法则对权重$W_ij$和$b$有
\begin{align}
\frac{\partial}{\partial W_{ij}^{(l)}} J(W,b; x, y) &= a^{(l)}_j \delta_i^{(l+1)} \\
\frac{\partial}{\partial b_{i}^{(l)}} J(W,b; x, y) &= \delta_i^{(l+1)}.
\end{align}


In [1]:
import numpy as np
import numpy.random as rd

In [11]:
# def activation function
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1.0-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1.0 - x**2

In [186]:
# 双层的前馈神经网络的BP算法。
class NeuralNetwork:
    # 默认激活函数为S型函数
    def __init__(self, layers, activation='sigmoid'):
        # 精度设置
        self.error_criterion = []
        # 初始化error数(随机设置一个比criterion大的数),假如小于criterion，即停止迭代
        self.error = 1
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_prime
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_prime

        # Set weights
        self.weights = []
        # Layer 设置
        for i in range(1, len(layers) - 1):
            # 初始权重介于-1,1之间，生成一特征数量*神经元数量的权重矩阵，+1表示常数项对应的权重。
            r = 2*np.random.random((layers[i-1] + 1, layers[i] )) -1
            self.weights.append(r)
        # output layer - random((2+1, 1)) : 3 x 1
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)

    def fit(self, X, y, learning_rate=1, epochs=10000,error_criterion=0.01):
        self.error_criterion = error_criterion
        # epochs为迭代次数上限，最终选择在误差小于criterion后或者迭代100000次后，停止。         
        for k in range(epochs):
            if k % 1000 == 0: 
                print('epochs:', k)
            
            # 计算mean squared error，若小于criterion即停止更新。
            prediction = self.predict(X)
            mean_square_error = ((y-prediction)**2).mean()
            self.error = mean_square_error
            if mean_square_error < error_criterion:
                break
            # 随机选取元素做更新
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                    dot_value = np.dot(np.hstack((1,a[l])), self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)

            errors = y[i] - a[-1]
            
            deltas = [errors * self.activation_prime(a[-1])]
            
            #生成倒序delta
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(np.dot(self.weights[l][1:],deltas[-1]).dot(self.activation_prime(a[l])))

            deltas.reverse()

            # 更新权重
            for i in range(len(self.weights)):
                layer = np.atleast_2d(np.hstack((1,a[i])))
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)
            
    # 由输入和参数计算神经网络输出值
    def predict(self, X): 
        tmp = X
        for l in range(0, len(self.weights)):
            ones = np.ones((tmp.shape[0],1))
            dots =np.dot(np.hstack((ones, tmp)), self.weights[l])
            tmp = self.activation(dots)
        return tmp
       
    def mean_square_error(self,X,y):
        return ((y-self.predict(X))**2).mean()

# 第二题
对XOR运算进行训练,但结果不是很理想.
即使在迭代9000次,(之前尝试国100000次的情形),最终结果仍然停留在0.5处.不明白原因.

In [187]:
# Test for XOR
nn = NeuralNetwork([2, 3, 1])

X = np.array([[0, 0],[0, 1],[1, 0],[1, 1]])

y = np.array([0, 1, 1, 0])

nn.fit(X, y)


epochs: 0
epochs: 1000
epochs: 2000
epochs: 3000
epochs: 4000
epochs: 5000
epochs: 6000
epochs: 7000
epochs: 8000
epochs: 9000


In [188]:
nn.predict(X)

array([[ 0.3429196 ],
       [ 0.35613153],
       [ 0.33280655],
       [ 0.33445151]])

In [189]:
nn.error

0.26937145976643928

# Test for Question 3
使用2-m-1结构的前馈网络逼近$[0,1]^2$上的连续函数$y=f(x)=1/(1+\sqrt(x_1 ^2+x_2 ^2))$


In [38]:
# train_fun for numpy array
def train_fun(x):
    return 1/(1+(x*x).sum(1))

In [39]:
# 1*1 内的均匀格点
data=[]
for i in range(100):
    for j in range(100):
        data.append([i,j])
data = np.array(data)
data = 0.005 + data/100

In [40]:
# 随机选取5000个
train_loc = rd.choice(10000,5000,replace = False)
tmp = set(train_loc)
tmp2 = set(range(10000))
tmp3 = tmp2.difference_update(tmp)
training_data = data[train_loc]
testing_data = data[list(tmp3)]

此处不知道为什么，速度很慢。始终难以收敛到要求。（以及达到了，但没有正常break）

In [148]:
testnn = NeuralNetwork([2,5,1])
y = train_fun(training_data)
testnn.fit(training_data,y)

epochs: 0


KeyboardInterrupt: 

In [152]:
testnn.weights

[array([[-0.50325027, -0.80837581, -0.71209436,  0.54072039,  0.15305065],
        [-1.76344064, -1.44920066, -0.89819535, -1.91194376, -1.52794614],
        [-2.01661244, -1.75081518, -1.8705454 , -0.86569561, -1.70634533]]),
 array([[-0.05322174],
        [ 0.52131733],
        [ 0.6899994 ],
        [ 0.60008547],
        [ 0.0413903 ],
        [ 1.7299185 ]])]

此处显示对训练样本的mean squared error已经小于error criterion

In [53]:
testnn.error

0.0057749077749055504

In [153]:
testnn.predict(training_data)

array([[ 0.5693499 ],
       [ 0.60821122],
       [ 0.65583173],
       ..., 
       [ 0.57624142],
       [ 0.59523835],
       [ 0.57949126]])

In [178]:
predition = testnn.predict(testing_data)

In [179]:
tmp = predition - train_fun(testing_data)

对样本的预测mean squared error远远大于训练样本，显示由过拟合的现象。这也与损失函数中并没有添加正则项有关。

In [181]:
tmp**2

0.030990427107420407

In [ ]:
def mean_squared_error()

In [ ]:
# 对不同的m值分别测试，计算predicting error并记录。
error =[]
for m in range(3,10):
    nn = NeuralNetwork[[2,m,1]]
    nn.fit(training_data,y)
    error.append(nn.mean_sqaured_error(testing_data,train_fun(testing_data)))
    